In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

import math
import datetime
import time

import requests
import json
from addict import Dict
import jmespath
import numpy as np
import pandas as pd
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
def do_request(func,**kwargs):
    response= func(**kwargs)
    if response.status_code==200:
        return response
    elif response.status_code==429:
        wait_after_seconds=int(response.headers["Retry-After"])
        print(f"429 Error\tWaiting seconds: {wait_after_seconds+1}")
        time.sleep(wait_after_seconds+1)
        return do_request(func,**kwargs)
    else:
        raise ValueError(f"{response.status_code}\t{response.text}")

In [ ]:
il_kodlari=[i for i in range(1,82)]
def get_ilceler(il_kodu):
    url = f"https://api.emlakjet.com/geolocation/location/{il_kodu}?fields=name,parentId,center,slug&includeChildren=true&includeRelated=false"

    payload = {}
    headers = {
      'authority': 'api.emlakjet.com',
      'pragma': 'no-cache',
      'cache-control': 'no-cache',
      'accept': 'application/json, text/plain, */*',
      'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36',
      'dnt': '1',
      'origin': 'https://www.emlakjet.com',
      'sec-fetch-site': 'same-site',
      'sec-fetch-mode': 'cors',
      'sec-fetch-dest': 'empty',
      'referer': 'https://www.emlakjet.com/satilik-konut/istanbul/',
      'accept-language': 'en,tr-TR;q=0.9,tr;q=0.8'
    }

    response = do_request(requests.request,method="GET",url=url, headers=headers, data = payload)
    return Dict(json.loads(response.text))

In [ ]:
res=[]
for il_kodu in il_kodlari:
    item=get_ilceler(il_kodu)
    for temp in item.data.children:
        temp['il_adi']=item.data.name
        res.append(temp)

In [ ]:
ilce_location_df=pd.DataFrame.from_records(res)[['parentId','il_adi', 'id','name','center','slug']]
ilce_location_df.columns=['endeksa_il','emlakjet_il_ad','endeksa_ilce','emlakjet_ilce_ad','latlot','slug']
ilce_location_df.sort_values(by=['endeksa_il', 'endeksa_ilce'], inplace=True)
ilce_location_df.reset_index(inplace=True)
ilce_location_df.drop('index', axis=1,inplace= True)
#ilce_location_df.to_csv('/data/sipai/Cumhur/il_ilce_kod.csv', sep= '~', encoding= 'ISO-8859-9')

In [ ]:
def get_il_demography(CityId):
    if not isinstance(CityId,str):
        CityId=str(CityId)
    url = "https://app.endeksa.com/demography"

    payload = "{\"Level\":1,\"CityId\":"+CityId+"}"
    headers = {
      'authority': 'app.endeksa.com',
      'pragma': 'no-cache',
      'cache-control': 'no-cache',
      'accept': 'application/json, text/plain, */*',
      'dnt': '1',
      'accept-language': 'tr',
      'user-agent': '4fe54daf-c9bd-441f-8e98-24e9d480dc33',
      'content-type': 'application/json;charset=UTF-8',
      'origin': 'https://www.endeksa.com',
      'referer': 'https://www.endeksa.com/',
    }

    response = do_request(requests.request,method="POST",url = url, headers=headers, data = payload)
    return Dict(json.loads(response.text))

In [ ]:
endeksa_il_ilce = pd.DataFrame()
for i in range(1,82):
    il_ilce_ = pd.DataFrame(sorted([(d_['CityId'], d_['CountyId']) for d_ in get_il_demography(i)['SubRegionals']]))
    il_ilce_.columns= ['endeksa_il', 'endeksa_ilce']
    endeksa_il_ilce= pd.concat((endeksa_il_ilce, il_ilce_))

In [1]:
# Burada amaç ENDEKSA ile EMLAKJETten gelen IL-ILCE kodu bilgilerini birleştirmek, neden birleştiriyoruz? Çünkü EMLAKJET verisinde SLUG ve LATLOG bilgisi var ve IL-AD ILCE-AD degerleri daha düzgün

In [ ]:
"""Aşağıdaki donusum de gerekli 
emlakjet_ilce_kod degeri modifiye edilmesi gerekiyor.
        case 
           when p.emlakjet_ilce_kod = 2109 then 2105  --Artvin Kemalpaşa
           when p.emlakjet_ilce_kod = 2093 then 2076  --Aydın Efeler
           when p.emlakjet_ilce_kod = 2090 then 2078  --Balıkesir Karesi
           when p.emlakjet_ilce_kod = 2091 then 2077  --Balıkesir Altıeylül
           when p.emlakjet_ilce_kod = 2092 then 2079  --Denizli Merkezefendi
           when p.emlakjet_ilce_kod = 2108 then 2090  --Muğla Seydikemer
           when p.emlakjet_ilce_kod = 2104 then 2095  --Tekirdağ Kapaklı 
           when p.emlakjet_ilce_kod = 2105 then 2096  --Tekirdağ Süleymanpaşa
           when p.emlakjet_ilce_kod = 2095 then 2091  --Şanlıurfa Eyyübiye
           when p.emlakjet_ilce_kod = 2096 then 2092  --Şanlıurfa Haliliye
           when p.emlakjet_ilce_kod = 2106 then 2093  --Şanlıurfa Karaköprü
           when p.emlakjet_ilce_kod = 2110 then 2106  --Aksaray Sultanhanı
           else p.emlakjet_ilce_kod
        end """
        